# 1. 생각흐름: 어떻게 하면 돈을 더 벌 수 있을까?
    # 1. 누가 (Matrix1, Matrix2)
    # 2. 어떤 상품을 많이 샀지? 
    # 3. 우리는 그 상품을 Sourcing하면 되나?

### 2. 구매 소요 시간 분석     
      1-1. 가입일로부터 구매일까지 전환된 소요시간 분석     
      
          ㄴ 최근 3개월 전체 분석 / 각 월 별 분석  
          
          ㄴ 분기 경향 / 월 별 변동사항 확인 -> 트렌드 분석     
          
          ex. 가입 1일 이내 전환 유저 : 10% -> 15% 상승 성과

In [1]:
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
# plt.style.use('seaborn-whitegrid')
import os
from datetime import date

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [3]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정
#plt.style.use("ggplot")


## 1. 유저의 첫번째 사용일 토대로 유저별 코호트 그룹 설정하기

In [4]:
df = pd.read_excel("클로젯셰어_과제DB 대체 자료.xlsx")

In [5]:
matrix_2 = df.copy()

In [6]:
matrix_2.set_index("USER", inplace=True)

In [7]:
matrix_2

,brandnm,goodsnm,reg_date,order_date,p_type,sub_type,app_yn
USER,,,,,,,
a161237,Gucci,flower dionysus shoulder bag beige,2019-12-06,2021-01-15,7days,BA,Y
a1140859,El estilo de Claire,beads cropped jacket black,2021-01-02,2021-01-15,4days,O,NaN
a1140859,Danha,embroidery point oriental mini skirt blue,2021-01-02,2021-01-15,4days,B,NaN
a1140859,Danha,graphics pattern top pink,2021-01-02,2021-01-15,4days,T,NaN
a1140740,Burberry London,buckle basic pattern zipup-jumper navy,2021-01-01,2021-01-15,7days,O,Y
...,...,...,...,...,...,...,...
a119721,Yves Saint Laurent,logo point clutch bag pink,2019-04-10,2021-03-01,7days,BA,Y
a1146069,Prada,saffiano lux M gray,2021-02-11,2021-03-01,7days,BA,Y
a1147848,Burberry,basic pattern sweater beige,2021-02-28,2021-03-01,7days,T,Y


In [8]:
FirstOrder = matrix_2.groupby(matrix_2.index)["order_date"].min()

In [9]:
matrix_2['FirstOrder'] = FirstOrder

In [10]:
matrix_2

,brandnm,goodsnm,reg_date,order_date,p_type,sub_type,app_yn,FirstOrder
USER,,,,,,,,
a161237,Gucci,flower dionysus shoulder bag beige,2019-12-06,2021-01-15,7days,BA,Y,2021-01-15
a1140859,El estilo de Claire,beads cropped jacket black,2021-01-02,2021-01-15,4days,O,NaN,2021-01-15
a1140859,Danha,embroidery point oriental mini skirt blue,2021-01-02,2021-01-15,4days,B,NaN,2021-01-15
a1140859,Danha,graphics pattern top pink,2021-01-02,2021-01-15,4days,T,NaN,2021-01-15
a1140740,Burberry London,buckle basic pattern zipup-jumper navy,2021-01-01,2021-01-15,7days,O,Y,2021-01-15
...,...,...,...,...,...,...,...,...
a119721,Yves Saint Laurent,logo point clutch bag pink,2019-04-10,2021-03-01,7days,BA,Y,2021-03-01
a1146069,Prada,saffiano lux M gray,2021-02-11,2021-03-01,7days,BA,Y,2021-03-01
a1147848,Burberry,basic pattern sweater beige,2021-02-28,2021-03-01,7days,T,Y,2021-03-01


In [31]:
matrix_2.reset_index(inplace=True)

In [33]:
matrix_2["USER"].value_counts()

a1149461    11
a1149583    10
a1142677     9
a189404      7
a1143701     7
            ..
a1149819     1
a1150434     1
a1142352     1
a1150975     1
a1151181     1
Name: USER, Length: 1234, dtype: int64

In [36]:
matrix_2[matrix_2["USER"] == "a1149461"]

,USER,brandnm,goodsnm,reg_date,order_date,p_type,sub_type,app_yn,FirstOrder
1148,a1149461,Alice & Olivia,see-through lace detail blouse black,2021-03-15,2021-03-17,4days,T,NaN,2021-03-17
1149,a1149461,Sandro,embroidery detail lace skirt multi,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17
1150,a1149461,Time,draping detail midi skirt gray,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17
1151,a1149461,El estilo de Claire,ruffle detail see-through sleeve blouse black,2021-03-15,2021-03-17,4days,T,NaN,2021-03-17
1152,a1149461,It michaa,pocket basic pattern skirt green,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17
1153,a1149461,roajeu,ruffle basic pattern mermaid trumpet-skirt green,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17
1154,a1149461,Alexander Wang,basic pattern skirt grey,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17
1155,a1149461,Christian Dior,basic pattern sweater burgundy,2021-03-15,2021-03-17,4days,T,NaN,2021-03-17
1156,a1149461,MSGM,basic pattern top black,2021-03-15,2021-03-17,4days,T,NaN,2021-03-17
1157,a1149461,Fayewoo,ruffle lace pattern mermaid trumpet-skirt red,2021-03-15,2021-03-17,4days,B,NaN,2021-03-17


In [37]:
df[df["USER"] == "a1149461"]

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
1148,Alice & Olivia,see-through lace detail blouse black,a1149461,2021-03-15,2021-03-17,4days,T,NaN
1149,Sandro,embroidery detail lace skirt multi,a1149461,2021-03-15,2021-03-17,4days,B,NaN
1150,Time,draping detail midi skirt gray,a1149461,2021-03-15,2021-03-17,4days,B,NaN
1151,El estilo de Claire,ruffle detail see-through sleeve blouse black,a1149461,2021-03-15,2021-03-17,4days,T,NaN
1152,It michaa,pocket basic pattern skirt green,a1149461,2021-03-15,2021-03-17,4days,B,NaN
1153,roajeu,ruffle basic pattern mermaid trumpet-skirt green,a1149461,2021-03-15,2021-03-17,4days,B,NaN
1154,Alexander Wang,basic pattern skirt grey,a1149461,2021-03-15,2021-03-17,4days,B,NaN
1155,Christian Dior,basic pattern sweater burgundy,a1149461,2021-03-15,2021-03-17,4days,T,NaN
1156,MSGM,basic pattern top black,a1149461,2021-03-15,2021-03-17,4days,T,NaN
1157,Fayewoo,ruffle lace pattern mermaid trumpet-skirt red,a1149461,2021-03-15,2021-03-17,4days,B,NaN


In [11]:
#How_Long = matrix_2["FirstOrder"] - matrix_2["reg_date"]

In [12]:
#First_Day = matrix_2["FirstOrder"].min()

In [13]:
#Last_Day = matrix_2["FirstOrder"].max()

In [14]:
#First_Day

In [15]:
#Last_Day

In [16]:
#Last_Day - First_Day

In [17]:
# week = []
# for x in range(len(matrix_2)):
#     if matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6):
#         week.append("1")
#     elif First_Day + pd.DateOffset(days=6) < matrix_2["FirstOrder"].iloc[x]  <= First_Day + pd.DateOffset(days=6*2):
#         week.append("2")
#     elif First_Day + pd.DateOffset(days=6*2) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*3):
#         week.append("3")
#     elif First_Day + pd.DateOffset(days=6*3) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*4):
#         week.append("4")
#     elif First_Day + pd.DateOffset(days=6*4) < matrix_2["FirstOrder"].iloc[x] <=  First_Day + pd.DateOffset(days=6*5):
#         week.append("5")
#     elif First_Day + pd.DateOffset(days=6*5) < matrix_2["FirstOrder"].iloc[x] <=  First_Day + pd.DateOffset(days=6*6):
#         week.append("6")
#     elif First_Day + pd.DateOffset(days=6*6) < matrix_2["FirstOrder"].iloc[x] <=  First_Day + pd.DateOffset(days=6*7):
#         week.append("7")
#     elif First_Day + pd.DateOffset(days=6*7) < matrix_2["FirstOrder"].iloc[x] <=  First_Day + pd.DateOffset(days=6*8):
#         week.append("8")
#     elif First_Day + pd.DateOffset(days=6*8) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*9):
#         week.append("9")
#     elif First_Day + pd.DateOffset(days=6*9) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*10):
#         week.append("10")
#     elif First_Day + pd.DateOffset(days=6*10) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*11):               
#         week.append("11")
#     elif First_Day + pd.DateOffset(days=6*11) <  matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*12):
#         week.append("12")
#     elif First_Day + pd.DateOffset(days=6*12) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*13):
#         week.append("13")
#     elif First_Day + pd.DateOffset(days=6*13) < matrix_2["FirstOrder"].iloc[x] <= First_Day + pd.DateOffset(days=6*14):
#         week.append("14")
#     else:
#         week.append("15")
        

In [18]:
#matrix_2["First_Week"] = week

In [19]:
#matrix_2.drop(["brandnm","goodsnm","reg_date","p_type","sub_type","app_yn","FirstOrder"], axis=1,inplace=True)

In [20]:
#matrix_2.reset_index(inplace=True)

In [21]:
#matrix_2["First_Week"] = matrix_2["First_Week"].astype(int)

In [22]:
#matrix_2.sort_values(by="First_Week").head(50)

## 2. 코호트 그룹과 주문기간으로 데이터 집계하기